In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('journals_shot.tsv',sep = '\t') 
df

,id,userId,eventTime,eventCode,content,additional
0,b25c7564-b084-42ef-90f7-ce8466b4560e,cf943c19-a2df-447b-9d26-77727d7c555b,2019-08-09 17:31:01.298,SignedInEvent,NaN,"{""utmParams"": null}"
1,17f45c55-2a8c-4b05-8d6f-651899a928cf,ca036cd8-dc22-4377-9fa4-fe073313e464,2019-08-09 17:31:08.639,ObjectViewedJournalEvent,"{""content"": {""identifier"": ""67fe6e4e-1e8c-4ab0...","{""utmParams"": null}"
2,b1e69af1-4c4e-4208-bf6c-8dc03ae6eb9b,ca036cd8-dc22-4377-9fa4-fe073313e464,2019-08-09 17:31:17.066,SearchExecutedJournalEvent,"{""docRequest"": {""skip"": 0, ""limit"": 0, ""query""...","{""utmParams"": null}"
3,5382f088-cf0c-419e-a14e-0466aa0452e2,ca036cd8-dc22-4377-9fa4-fe073313e464,2019-08-09 17:59:51.295,SignedOutEvent,NaN,"{""utmParams"": null}"
4,32dbe698-8d51-4148-9017-5137e75bac23,ca036cd8-dc22-4377-9fa4-fe073313e464,2019-08-09 17:59:51.948,SearchExecutedJournalEvent,"{""docRequest"": {""skip"": 0, ""limit"": 0, ""query""...","{""utmParams"": null}"
...,...,...,...,...,...,...
59995,9857c929-f728-4bb8-8483-7c7958426b94,ca036cd8-dc22-4377-9fa4-fe073313e464,2019-11-21 07:57:53.656,SearchExecutedJournalEvent,"{""docRequest"": {""skip"": 0, ""limit"": 0, ""query""...","{""utmParams"": null}"
59996,4aaa559d-c2be-4fa3-a440-4a1ce270f7cf,ca036cd8-dc22-4377-9fa4-fe073313e464,2019-11-21 07:57:57.881,SearchExecutedJournalEvent,"{""docRequest"": {""skip"": 0, ""limit"": 10, ""query...","{""utmParams"": null}"
59997,e4d6f7f6-4029-49dd-8ab8-e94d88366a9b,NaN,2019-11-21 08:22:21.476,RedirectedFromExternalWithUtm,NaN,"{""utmParams"": {""markerTypeToValue"": {""utm_sour..."
59998,aed5bd35-f5fd-4b92-a013-6cf8248d3a08,ae9217d9-0f9f-48cb-8abe-b114c20b98e0,2019-11-21 08:25:47.155,UserStartActivity,NaN,"{""utmParams"": null}"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          60000 non-null  object
 1   userId      55603 non-null  object
 2   eventTime   60000 non-null  object
 3   eventCode   60000 non-null  object
 4   content     40809 non-null  object
 5   additional  60000 non-null  object
dtypes: object(6)
memory usage: 2.7+ MB


In [7]:
# просмотр даных переменной content, чтобы понять нужно ли преобразовывать json и использовать переменную в решении задачи
df.content.iat[1]

'{"content": {"identifier": "67fe6e4e-1e8c-4ab0-9943-d532fc02de6e", "objectType": "profile"}, "jsonFormatterCode": "ObjectViewedJournalEvent"}'

In [8]:
df.additional.iat[59997]

'{"utmParams": {"markerTypeToValue": {"utm_source": "portal", "utm_content": "searchbar"}}}'

По-моему, чтобы ответить на поставленные вопросы нет необходимости в применении content и additional. Безусловно они могут дать дополнительную информацию, но на первый взгляд можно продолжить без них

In [9]:
# поиск количества уникальных пользователей
print('Количество уникальных пользователей:', len(df['userId'].unique()))

Количество уникальных пользователей: 2262


In [13]:
# Для изучения данных проверим количество пропущенных значений
# Функция для подсчета пропущенных значений
def missing_values_table(df):
        # всего N/A
        mis_val = df.isnull().sum()
        
        # Процент N/A
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Таблица с результатом
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Переименование столбцов
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Сортировка по уменьшению
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        
        print ("Выбранный набор данных имеет " + str(df.shape[1]) + " переменных.\n"      
            "Всего " + str(mis_val_table_ren_columns.shape[0]) +
              " имеют пропущенные значения.")
        
        
        return mis_val_table_ren_columns

In [14]:
missing_values_table(df)

Выбранный набор данных имеет 6 переменных.
Всего 2 имеют пропущенные значения.


,Missing Values,% of Total Values
content,19191,32.0
userId,4397,7.3


32% пропущенных значений могут значительно повлиять на качество выводов, однако в рамках задачи переменная content не будет использована мной, поэтому ее можно как исключить из набора данных, так и ничего не делать

In [5]:
# какие действия пользователи совершают чаще всего? 
df2 = df.groupby(['eventCode'])['eventCode'].count().sort_values(ascending=False)
print(df2)

eventCode
SearchExecutedJournalEvent       33184
RedirectedFromExternalWithUtm     9629
UserStartActivity                 8188
ObjectViewedJournalEvent          7112
CatalogListViewedEvent             564
ProfileListViewedEvent             524
SearchQueryExpandedEvent           283
CatalogViewedEvent                 158
BookmarksViewedEvent                74
CatalogItemUpdatedEvent             34
BookmarkAddedEvent                  33
FeedbackSent                        30
OrganizationListViewedEvent         28
SignedInEvent                       24
UserRoleAssignation                 21
SignedOutEvent                      19
SignInFailed                        17
ErrorFound                          15
JournalDownloaded                   14
CatalogItemCreatedEvent             13
RoleMappingRulesUpdated             11
BookmarkRemovedEvent                11
CatalogDeletedEvent                  7
UserStateChangedEvent                4
CatalogItemDeletedEvent              3
Name: eventCode

Таким образом, пользователи совершают следующие действия чаще всего: SearchExecutedJournalEvent	"Выполнен поиск", RedirectedFromExternalWithUtm	"Пользователь перенаправлен с внешнего ресурса", UserStartActivity	"Пользователь активен", ObjectViewedJournalEvent	"Просмотрен объект" 

In [17]:
df['eventCode'].count()

60000

In [23]:
# расчет среднего количества событий на одного пользователя
n_user_event = 60000/2262
print(n_user_event)

26.52519893899204


Спасибо Вам за уделенное моей работе время. Наверное, Вы ожидали большего, однако в условиях сильно ограниченного времени, я решила, что лучше пришлю часть выполненного задания, чем не отправлю ничего. Надеюсь, у меня еще будет шанс показать свои знания.